In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pulp import *

In [5]:
from docplex.mp.model import Model

In [3]:
import cplex

## Reading in Dataset

In [225]:
CUT_OFF = 2500


### Fixed Cost of each WTP

In [226]:
fixed_cost = pd.read_excel("../dataset2/fixed_cost.xlsx", index_col=0)
fixed_cost.head()

,Fixed Cost
WTP,
Ampang Intake,11638
Batang Kali,11011
Bernam River Head,11602
WTP1,4899
WTP2,11773


### Capacity of each WTP

In [227]:
capacity = pd.read_excel("../dataset2/capacity.xlsx", index_col=0)
capacity.head()

,Capacity
WTP,
Ampang Intake,2.641160e+07
Batang Kali,2.858907e+07
Bernam River Head,2.993616e+07
WTP1,2.728402e+07
WTP2,2.898516e+07


In [228]:
capacity.loc["WTP1", "Capacity"]

27284019.78365

### Distribution Loss

In [229]:
distribution_loss = pd.read_excel("../dataset2/distribution_loss.xlsx", index_col=0)
distribution_loss.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.31838,0.34819,0.23241,0.26049,0.34462,0.28837,0.30351,0.28379,0.33333,0.30272,...,0.29068,0.28830,0.29829,0.35956,0.26697,0.30082,0.27045,0.28345,0.27237,0.33081
DMZ002,0.30544,0.32145,0.29041,0.26520,0.30354,0.25078,0.31171,0.26908,0.24993,0.34083,...,0.31445,0.24484,0.35810,0.34966,0.31344,0.25027,0.29574,0.27309,0.32620,0.30501
DMZ003,0.32782,0.29775,0.30906,0.33927,0.31701,0.29337,0.26027,0.31059,0.30448,0.32909,...,0.24614,0.32459,0.23055,0.31726,0.31519,0.31940,0.29601,0.28883,0.33544,0.26115
DMZ004,0.35452,0.28941,0.25286,0.30884,0.32873,0.27913,0.26008,0.33860,0.32277,0.28385,...,0.32659,0.26638,0.28412,0.34979,0.26636,0.28850,0.28361,0.29359,0.25466,0.26176
DMZ005,0.28498,0.28067,0.29266,0.32907,0.29612,0.33973,0.24868,0.34067,0.25028,0.28086,...,0.31274,0.31326,0.29827,0.27726,0.29009,0.23659,0.27612,0.35060,0.32065,0.27252


In [230]:
distribution_loss.loc["DMZ001", "WTP1"]

0.26049

### Links between WTP and DMZ

In [231]:
linkage = pd.read_excel("../dataset2/linkage2.xlsx", index_col=0)
linkage.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,1,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
DMZ002,1,1,1,1,0,1,1,1,1,1,...,1,0,1,1,1,0,1,1,0,0
DMZ003,1,1,1,1,1,1,1,1,0,1,...,1,1,0,1,1,1,1,1,0,1
DMZ004,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,1,1,1,1,0,1
DMZ005,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,1,1,0


### Variable Cost for each combination of (DMZ, WTP)

In [232]:
variable_cost = pd.read_excel("../dataset2/variable_costs.xlsx", index_col=0)
variable_cost.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,9,14,9,12,24,13,21,20,8,10,...,21,23,11,9,17,7,19,18,5,22
DMZ002,12,15,23,11,18,23,17,7,7,22,...,9,9,10,23,13,18,9,15,9,11
DMZ003,5,20,19,8,9,17,7,12,18,9,...,21,21,19,15,15,5,6,16,11,21
DMZ004,22,14,15,6,12,8,9,19,17,11,...,15,9,9,24,20,8,19,13,15,5
DMZ005,22,10,7,21,22,12,21,20,9,18,...,8,14,18,23,20,14,7,5,6,5


### Transport Cost for each combination of (DMZ, WTP)

In [233]:
# transport_cost = pd.read_excel("../dataset/freight_costs.xlsx", index_col=0)
# transport_cost.head()

### Demand for each DMZ

In [234]:
demand = pd.read_excel("../dataset2/demand_chisq.xlsx", index_col=0)
demand.head()

,Demand
DMZ001,160044.62961
DMZ002,150056.74766
DMZ003,141891.56807
DMZ004,194644.00762
DMZ005,126901.55527


In [235]:
demand = demand.iloc[:CUT_OFF,:]

In [236]:
demand.loc["DMZ001", "Demand"]

160044.62961

In [237]:
# demand["Demand"] = pd.to_numeric(demand["Demand"], downcast="float")
# demand.head()

## Optimisation

In [238]:
# List of all the WTPs
wtp = list(capacity.index)

# List of all the DMZs
dmz = list(demand.index)

# List of (DMZ, WTP) pairs
dmz_wtp_pairs = [(d, w) for d in dmz for w in wtp]

In [239]:
print(wtp[:5])
print("Number of WTP:", len(wtp) )

print(dmz[:5])
print("Number of DMZ:", len(dmz) )

print("Number of DMZ-WTP Pairs:", len(dmz_wtp_pairs))

['Ampang Intake', 'Batang Kali', 'Bernam River Head', 'WTP1', 'WTP2']
Number of WTP: 45
['DMZ001', 'DMZ002', 'DMZ003', 'DMZ004', 'DMZ005']
Number of DMZ: 2500
Number of DMZ-WTP Pairs: 112500


In [240]:
demand.loc[dmz[0], "Demand"]

160044.62961

In [241]:
# Creating the Linear Optimisation Class
# model = LpProblem("Optimising water supply", LpMinimize)

model = Model(name="Optimising water supply")

In [242]:
# Creating Decision Variables
# output = LpVariable.dicts("Volume", dmz_wtp_pairs, lowBound=0, upBound=None, cat='continuous')

output = model.continuous_var_dict(keys=dmz_wtp_pairs, name="Volume")

# output = {}
# for var in dmz_wtp_pairs:
#     ouptut[var] = model.continuous_var(name=str(var))



In [243]:
output[("DMZ001", "WTP1")]

docplex.mp.Var(type=C,name='Volume_DMZ001_WTP1')

In [244]:
# # Adding Constraints

## Meet demand for each DMZ
for d in dmz:
    model.add_constraint(model.sum([ (output[(d, w)] * (1-distribution_loss.loc[d, w]) * (linkage.loc[d, w])) for w in wtp]) >= demand.loc[d, "Demand"])

## Within the WTP capacity
for w in wtp:
    model.add_constraint(model.sum([output[(d, w)] for d in dmz]) <= capacity.loc[w, "Capacity"])

In [245]:
# Define the Objective Function

## Without transport cost
model.minimize(
     model.sum([fixed_cost.loc[w, "Fixed Cost"] * 1000 for w in wtp]) + \
     model.sum([(variable_cost.loc[d, w]) * output[(d, w)] for d in dmz for w in wtp])
)


## With transport cost
# model += \
#      lpSum([fixed_cost.loc[w, "Fixed Cost"] * 1000 for w in wtp]) + \
#      lpSum([(variable_cost.loc[d, w] + transport_cost.loc[d, w]) * output[(d, w)] for d in dmz for w in wtp])

In [246]:
model.print_information()

Model: Optimising water supply
 - number of variables: 112500
   - binary=0, integer=0, continuous=112500
 - number of constraints: 2545
   - linear=2545
 - parameters: defaults
 - objective: minimize
 - problem type is: LP


In [247]:
solution = model.solve()

In [248]:
if solution is None:
    print("Model is infeasible")
else:
    print("Optimal solution found")
    print("Total Costs = {:,.2f} ($/Month)".format(solution.objective_value))

Optimal solution found
Total Costs = 3,329,741,038.85 ($/Month)


In [249]:
# ## Meet demand for each DMZ
# for d in dmz:
#     for w in wtp:
#         # No linkage constraint
#         if linkage.loc[d, w] == 0 :
#             model += output[(d,w)] == 0
    
#     model += LpConstraint(
#         lpSum([ (output[(d, w)] * (1-distribution_loss.loc[d, w])) for w in wtp]),
#         sense=LpConstraintGE,
#         rhs= demand.loc[d, "Demand"],
#         name=d
#         )


In [250]:
# # # Adding Constraints

# ## Meet demand for each DMZ
# for d in dmz:
#     model += lpSum([ (output[(d, w)] * (1-distribution_loss.loc[d, w]) * (linkage.loc[d, w])) for w in wtp]) >= demand.loc[d, "Demand"]

# ## Within the WTP capacity
# for w in wtp:
#     model += lpSum([output[(d, w)] for d in dmz]) <= capacity.loc[w, "Capacity"]

## No linkage constraint





# Adding Constraints


# ## Within the WTP capacity
# for w in wtp:
#     model += LpConstraint(
#         lpSum([output[(d, w)] for d in dmz]),
#         sense = LpConstraintLE,
#         rhs = capacity.loc[w, "Capacity"]*1000,
#         name = w.replace(" ", "")
#         )





In [251]:
# # # Solve the model
# solver = CPLEX_PY()
# solver.buildSolverModel(model)

# #Modify the solvermodel
# # solver.solverModel.parameters.timelimit.set(1200)

# #Solve P
# solver.callSolver(model)
# status = solver.findSolutionValues(model)

In [252]:
# # Solve the model
# model.solve(PULP_CBC_CMD(maxSeconds=1000, msg=1, fracGap=0))

In [253]:
# # Solve the model
# model.solve()

In [254]:
# # Solve the model with cplex
# model.solve(CPLEX_PY())

In [255]:
# print("Total Costs = {:,} ($/Month)".format(int(value(model.objective))))
# print('\n' + "Status: {}".format(LpStatus[model.status]))

In [256]:
# soln_dict = {i.name: i.varValue for i in model.variables()}

In [257]:
# not_satisfied = []

# for c in model.constraints.values():
#     c_dict = c.toDict()
#     # print(c_dict)

#     satisfied = False
    
#     LHS = sum([soln_dict[i['name']]*i['value'] for i in c_dict['coefficients']])
#     LHS = LHS + c_dict['constant']
    
#     if c_dict['sense'] == 0:
#         satisfied = (LHS == 0)
   
#     if c_dict['sense'] == -1:
#         satisfied = (LHS <= 0)
    
#     if c_dict['sense'] == 1:
#         satisfied = (LHS >= 0)
   
#     # print(c)
    
#     if satisfied:
#         #print('LHS: ', LHS)
#         print('is satisfied')
#     else:
#         #print('LHS: ', LHS)
#         print(c_dict["name"],'not satisfied', "LHS:", LHS)
#         not_satisfied.append(c_dict["name"])

## Results

In [258]:
dict_wtp = {}
dict_dmz = {}

In [259]:
# df = pd.DataFrame(0, index=dmz, columns = wtp)
# df

df = pd.DataFrame()
df

""


In [260]:
# combi = list(model.iter_variables())[0].name.replace("Volume_", "").replace("_", ",").split(",")
# curr_dmz = combi[0]
# curr_wtp = combi[1]
# volume = output[(curr_dmz, curr_wtp)].solution_value

In [261]:
# Getting the results
for v in model.iter_variables():
    # print(v.name, v.varValue)
    name = v.name.replace("Volume_", "").replace("_", ",")
    # print(name)

    combi = name.split(",")
    # print(combi[0])

    curr_dmz = combi[0]
    curr_wtp = combi[1]
    volume = v.solution_value

    # print("DMZ: ", dmz, " ", "WTP: ", wtp, " ", "Value: ", volume)

    df.loc[curr_dmz, curr_wtp] = volume


In [262]:
# Supply of water from each WTP to each DMZ (including distribution loss)
df

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,219953.313648,0.000000
DMZ002,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
DMZ003,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,208480.117646,0.0,0.0,0.000000,0.000000
DMZ004,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,263659.524843
DMZ005,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMZ2496,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,231131.791574,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
DMZ2497,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
DMZ2498,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
DMZ2499,0.0,0.0,0.0,266321.788435,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


In [263]:
# df.columns = ["Ampang Intake", "Batang Kali", "Bername River Head", "Sungai Tengi", "Wangsa Maju"]
# df = df[["Ampang Intake", "Batang Kali", "Bername River Head", "Wangsa Maju", "Sungai Tengi", ]]

In [264]:
# Supply of water from each WTP to each DMZ (including distribution loss)
# df

### Renaming the columns for distribution loss to match result df

In [265]:
col_list = df.columns.tolist()
print(col_list[:5])

temp_col_list = distribution_loss.columns.tolist()
# dist_col_list = [w.replace(" ", "") for w in temp_col_list]
dist_col_list = temp_col_list

print(dist_col_list[:5])

['Ampang Intake', 'Batang Kali', 'Bernam River Head', 'WTP1', 'WTP2']
['Ampang Intake', 'Batang Kali', 'Bernam River Head', 'WTP1', 'WTP2']


In [266]:
result_distribution_loss = distribution_loss.iloc[:CUT_OFF, :].copy()
result_distribution_loss.columns = dist_col_list

In [267]:
result_distribution_loss = result_distribution_loss[col_list]

### Accounting for distribution loss

In [268]:
useful_amount = 1- result_distribution_loss.values

In [269]:
temp = df.values * useful_amount
temp

array([[     0.     ,      0.     ,      0.     , ...,      0.     ,
        160044.62961,      0.     ],
       [     0.     ,      0.     ,      0.     , ...,      0.     ,
             0.     ,      0.     ],
       [     0.     ,      0.     ,      0.     , ...,      0.     ,
             0.     ,      0.     ],
       ...,
       [     0.     ,      0.     ,      0.     , ...,      0.     ,
             0.     ,      0.     ],
       [     0.     ,      0.     ,      0.     , ...,      0.     ,
             0.     ,      0.     ],
       [     0.     ,      0.     ,      0.     , ...,      0.     ,
             0.     ,      0.     ]])

In [270]:
temp.shape

(2500, 45)

In [271]:
# Supply of water from each WTP to each DMZ, after accounting for water loss
final_df = pd.DataFrame(temp, columns = wtp, index=dmz)
final_df.round(3)

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.0,160044.63,0.000
DMZ002,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.000
DMZ003,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,141891.568,0.0,0.0,0.00,0.000
DMZ004,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.00,194644.008
DMZ005,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DMZ2496,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,161017.963,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.000
DMZ2497,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.000
DMZ2498,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.000
DMZ2499,0.0,0.0,0.0,187727.565,0.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.000


In [272]:
# wtp

In [273]:
supply = final_df.sum(axis=1)
supply[:5]

DMZ001    160044.62961
DMZ002    150056.74766
DMZ003    141891.56807
DMZ004    194644.00762
DMZ005    126901.55527
dtype: float64

In [276]:
prob = []
print("       ", "Supply", "            ", "Demand")
for curr_dmz in dmz:
    a = supply[curr_dmz]
    b = demand.loc[curr_dmz, "Demand"]
    if not (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)) and a <= b:
        print(curr_dmz, supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
        prob.append(curr_dmz)

        Supply              Demand


In [277]:
# prob = []
# for curr_dmz in dmz:
#     # print(supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
#     # print(supply[curr_dmz] >= demand.loc[curr_dmz, "Demand"])
#     a = supply[curr_dmz]
#     b = demand.loc[curr_dmz, "Demand"]
#     if not (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)) and a <= b:
#         print(curr_dmz, supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
#         prob.append(curr_dmz)
#     # elif (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)):
#     #     print(True)
#     # else: 
#     #     print(supply[curr_dmz], ">=", demand.loc[curr_dmz, "Demand"], "?")
#     #     print(supply[curr_dmz] >= demand.loc[curr_dmz, "Demand"])

In [278]:
len(prob)

0

In [279]:
check_capacity = final_df.sum(axis=0)

In [280]:
prob_wtp = []
for curr_wtp in wtp:
    # print(curr_dmz, check_capacity[curr_wtp], "<=", capacity.loc[curr_wtp, "Capacity"], "?")
    # print(check_capacity[curr_wtp] <= capacity.loc[curr_wtp, "Capacity"])
    a = check_capacity[curr_wtp]
    b = capacity.loc[curr_wtp, "Capacity"]
    if not (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)) and a >= b:
        print(curr_dmz, check_capacity[curr_wtp], "<=", capacity.loc[curr_wtp, "Capacity"], "?")
        prob_wtp.append(curr_wtp)
    # elif (np.isclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)):
    #     print(True)
    # else: 
    #     # print(curr_dmz, check_capacity[curr_wtp], "<=", capacity.loc[curr_wtp, "Capacity"], "?")
    #     print(check_capacity[curr_wtp] <= capacity.loc[curr_wtp, "Capacity"])

In [281]:
len(prob_wtp)

0

In [282]:
df.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,219953.313648,0.000000
DMZ002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
DMZ003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,208480.117646,0.0,0.0,0.000000,0.000000
DMZ004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,263659.524843
DMZ005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


In [283]:
final_df.head()

,Ampang Intake,Batang Kali,Bernam River Head,WTP1,WTP2,WTP3,WTP4,WTP5,WTP6,WTP7,...,WTP33,WTP34,WTP35,WTP36,WTP37,WTP38,WTP39,WTP40,Wangsa Maju,Sungai Tengi
DMZ001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,160044.62961,0.00000
DMZ002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.00000
DMZ003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,141891.56807,0.0,0.0,0.00000,0.00000
DMZ004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,194644.00762
DMZ005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.00000


## Export results

In [284]:
# df.to_excel("../dataset2/result(2500)_without_loss.xlsx", index=True)
# final_df.to_excel("../dataset2/result(2500).xlsx", index=True)